In [14]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("DSC 400 Assignment 10") \
    .getOrCreate()

# TODO: Change these to point to a version on your local path
sample_libsvm_data_path = (r'C:\Users\12162\Documents\BU\Winter 2022\DSC 400\Week 10\sample_libsvm_data.txt')

RuntimeError: Java gateway process exited before sending its port number

In [2]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = spark.read.format('libsvm').load(sample_libsvm_data_path)

# TODO: Implement the remainder of the code from the binomial logistic regression example example
#________________________________________________________________________________________________

# Code to be implimented:
# _______________________

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print('Coefficients: ' + str(lrModel.coefficients))
print('Intercept: ' + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family='multinomial')

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print('Multinomial coefficients: ' + str(mlrModel.coefficientMatrix))
print('Multinomial intercepts: ' + str(mlrModel.interceptVector))

C:\Users\12162\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'spark' is not defined

In [3]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print('objectiveHistory:')
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a df and areaUnderROC.
trainingSummary.roc.show()
print('areaUnderROC: ' + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

NameError: name 'lrModel' is not defined

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

file_url = 'http://storage.googleapis.com/download.tensorflow.org/data/heart.csv'
df = pd.read_csv(file_url)

# TODO: Complete the remainder of the example code

In [5]:
df.shape

(303, 14)

In [6]:
# Creates dfs for training and validating the model
val_df = df.sample(frac=0.2, random_state=1337)
train_df = df.drop(val_df.index)

print(f'Train, Val sizes: {len(train_df), len(val_df)}')

Train, Val sizes: (242, 61)


In [7]:
# Creates TF dataset objects from dataframes
def df_to_ds(df):
    df = df.copy()
    
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    return ds

# Creates TF ds objects from dfs
train_ds = df_to_ds(train_df)
val_ds = df_to_ds(val_df)

In [8]:
# Batches the ds - Adds dimension to shape
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [9]:
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup


# Normoralizes and engineers features quantitative
def encode_numerical_feature(feature, name, ds):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a dataset that only yields our feature
    feature_ds = ds.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


# Normoralizes and engineers features qualitative
def encode_categorical_feature(feature, name, ds, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode='binary')

    # Prepare a ds that only yields our feature
    feature_ds = ds.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [10]:
# Categorizes Data by type
data_cats = {
    'continious': ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope'],
    'discrete': [ 'sex', 'cp', 'fbs', 'restecg', 'exang', 'ca'],
    'string': ['thal']
}

In [11]:
# The following code was written take up much less code-block realestate
# ...than it does in the tutorial. It also supports easier adjustments to
# ...input data now via editing data_cats{} values.
# ______________________________________________________________________

all_inputs = []
all_features = []

# Encodes numerically continious features
for label in data_cats.get('continious'):
    feat_input = keras.Input(shape=(1,), name=label)
    feature = encode_numerical_feature(feat_input, label, train_ds)
    
    all_inputs.append(feat_input)
    all_features.append(feature)

# Encodes numerically discrete features
for label in data_cats.get('discrete'):
    feat_input = keras.Input(shape=(1,), name=label, dtype='int64')
    feature = encode_categorical_feature(feat_input, label, train_ds, False)
    
    all_inputs.append(feat_input)
    all_features.append(feature)

# Encodes string features
for label in data_cats.get('string'):
    feat_input = keras.Input(shape=(1,), name=label, dtype='string')
    feature = encode_categorical_feature(feat_input, label, train_ds, True)
    
    all_inputs.append(feat_input)
    all_features.append(feature)

all_features = layers.concatenate(all_features)

x = layers.Dense(32, activation='relu')(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(all_inputs, output)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
8/8 [==============================] - 1s 40ms/step - loss: 0.8215 - accuracy: 0.4174 - val_loss: 0.7150 - val_accuracy: 0.4754
Epoch 2/50
8/8 [==============================] - 0s 3ms/step - loss: 0.7360 - accuracy: 0.5248 - val_loss: 0.6576 - val_accuracy: 0.5902
Epoch 3/50
8/8 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.6074 - val_loss: 0.6121 - val_accuracy: 0.6557
Epoch 4/50
8/8 [==============================] - 0s 4ms/step - loss: 0.5939 - accuracy: 0.6818 - val_loss: 0.5726 - val_accuracy: 0.7049
Epoch 5/50
8/8 [==============================] - 0s 4ms/step - loss: 0.5801 - accuracy: 0.7107 - val_loss: 0.5404 - val_accuracy: 0.7377
Epoch 6/50
8/8 [==============================] - 0s 4ms/step - loss: 0.5647 - accuracy: 0.7107 - val_loss: 0.5141 - val_accuracy: 0.7377
Epoch 7/50
8/8 [==============================] - 0s 4ms/step - loss: 0.5470 - accuracy: 0.7190 - val_loss: 0.4917 - val_accuracy: 0.7377
Epoch 8/50
8/8 [=================

In [13]:
sample = {
    'age': 60,
    'sex': 1,
    'cp': 1,
    'trestbps': 145,
    'chol': 233,
    'fbs': 1,
    'restecg': 2,
    'thalach': 150,
    'exang': 0,
    'oldpeak': 2.3,
    'slope': 3,
    'ca': 0,
    'thal': 'fixed',
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
preds = model.predict(input_dict)

print(
    f'This particular patient had a {100 * preds[0][0]} percent probability '
    'of having a heart disease, as evaluated by the deep learning model.'
)

1/1 [==============================] - 0s 296ms/step
This particular patient had a 34.514403343200684 percent probability of having a heart disease, as evaluated by the deep learning model.
